In [76]:
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

model = read_sbml_model('models/iMM904_progesterone.xml') #load model

model.medium #see composition of growth medium


model.metabolites.progesterone_c.summary

model.reactions.R02216.summary
#model.metabolites.progesterone_c.summary(fva=0.95)

#model.metabolites.get_by_id("progesterone_c").upper_bound = 1000



#model.reactions.EX_glc__D_e
#model.reactions.EX_glc__D_e.flux_expression
#model.optimize()
#model.reactions.EX_glc__D_e.flux


<bound method Reaction.summary of <Reaction R02216 at 0x140217b50>>

Maximum theoretical productivity and yield of progesterone on glucose (default medium):

In [72]:

with model:
    model.objective = model.metabolites.progesterone_c
    model.metabolites.get_by_id("progesterone_c").upper_bound = 1000
    progesterone_production = model.optimize().objective_value
    print('Maximum theoretical productivity of progesterone on glucose: ', progesterone_production, 'mmol/gDW*h')
    max_yield = progesterone_production / (-1 * model.reactions.EX_glc__D_e.flux)
    print('Maximum theoretical yield of progesterone on glucose: ', max_yield, 'mmol progesterone/mmol glucose')
    

TypeError: item in iterable cannot be '<class 'cobra.core.metabolite.Metabolite'>'

As seen xxxx

The xxxxxx